In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
from IPython.display import Audio

## Different attack styles 

The dataset contains 5 different attack types: 
* 0 - benign 
* 1 - Bruteforce attack 
* 2 - DoS attack  
* 3 - Web attack
* 4 - Inflitration attack 
* 5 - Botnet attack 
* 6 - DDoS attack 

In [2]:
def encode_target(label):
    if label=="Benign":
        return 0 # Benign network traffic
    if "Bruteforce" in label or "BruteForce" in label:
        return 1 # Bruteforce attack
    if "GoldenEye" in label or "Slowloris" in label:
        return 2 # DoS attack
    if "Web" in label or "XSS" in label or "SQL" in label:
        return 3 # Web attack
    if "Infilteration" in label:
        return 4 # Infiltration attack
    if "Bot" in label:
        return 5 # Botnet attack 
    if "DDoS" in label or "DDOS":
        return 6 # DDoS attack

In [3]:
def clean_data(data, time=False):
    # Drop Dst Port : REASON TO COME 
    data = data.drop("Dst Port", axis = 1)
    
    # Drop any rows with "Infinity" values or labels with the value "Label"
    data = data.drop(data[data.eq("Infinity").any(1)].index,axis=0)
    data = data.drop(data[data.eq("Label").any(1)].index,axis=0)
    
    # Percent of missing data 
    print("Missing Data:",round(data.isnull().sum().sum()/len(data)*100,3),"%") # Just over 0.2%, very minimal so we can remove this 
    data = data.dropna()

    # Percent of duplicated data
    print("Duplicated Data:", round(len(data[data.duplicated()])/len(data)*100,2),"%")
    data = data.drop_duplicates()

    if not time:
        # Dropping the timestamp feature 
        data = data.drop("Timestamp", axis = 1)

    # One hot encoding the protocol feature 
    dum_df = pd.get_dummies(data["Protocol"],prefix="protocol_")
    data = data.drop(["Protocol"],axis=1)
    data = pd.concat([data,dum_df],axis=1) 
    
    data["target"] = np.vectorize(encode_target)(data["Label"])
    data = data.drop("Label", axis = 1)

    return data 

### Brute-force Attacks

In [4]:
bruteforce = pd.read_csv("02-14-2018.csv")

In [5]:
bruteforce = clean_data(bruteforce)

Missing Data: 0.217 %
Duplicated Data: 21.65 %


In [6]:
bruteforce.head()

,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,Bwd Pkt Len Max,...,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,protocol__0,protocol__6,protocol__17,target
0,112641719,3,0,0,0,0,0,0.000000,0.000000,0,...,0,0,56320859.5,139.300036,56320958,56320761,1,0,0,0
1,112641466,3,0,0,0,0,0,0.000000,0.000000,0,...,0,0,56320733.0,114.551299,56320814,56320652,1,0,0,0
2,112638623,3,0,0,0,0,0,0.000000,0.000000,0,...,0,0,56319311.5,301.934596,56319525,56319098,1,0,0,0
3,6453966,15,10,1239,2273,744,0,82.600000,196.741237,976,...,0,0,0.0,0.000000,0,0,0,1,0,0
4,8804066,14,11,1143,2209,744,0,81.642857,203.745545,976,...,0,0,0.0,0.000000,0,0,0,1,0,0


In [7]:
bruteforce["target"].value_counts()

0    663094
1    156668
Name: target, dtype: int64

### Dos Attacks

In [8]:
dos = pd.concat([pd.read_csv("02-15-2018.csv",low_memory=False), pd.read_csv("02-15-2018.csv",low_memory=False)])

In [9]:
dos["Label"].value_counts()

Benign                   1992154
DoS attacks-GoldenEye      83016
DoS attacks-Slowloris      21980
Name: Label, dtype: int64

In [10]:
dos = clean_data(dos)

Missing Data: 0.469 %
Duplicated Data: 50.21 %


In [11]:
dos.head()

,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,Bwd Pkt Len Max,...,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,protocol__0,protocol__6,protocol__17,target
0,112641158,3,0,0,0,0,0,0.000000,0.000000,0,...,0,0,56320579.0,7.042784e+02,56321077,56320081,1,0,0,0
1,37366762,14,12,2168,2993,712,0,154.857143,254.855270,976,...,1601183,321569,11431221.0,3.644991e+06,15617415,8960247,0,1,0,0
2,543,2,0,64,0,64,0,32.000000,45.254834,0,...,0,0,0.0,0.000000e+00,0,0,0,1,0,0
3,112640703,3,0,0,0,0,0,0.000000,0.000000,0,...,0,0,56320351.5,3.669884e+02,56320611,56320092,1,0,0,0
4,112640874,3,0,0,0,0,0,0.000000,0.000000,0,...,0,0,56320437.0,7.198347e+02,56320946,56319928,1,0,0,0


In [12]:
dos["target"].value_counts()

0    987621
2     51740
Name: target, dtype: int64

### Web Attacks

In [13]:
web = pd.concat([pd.read_csv("02-22-2018.csv",low_memory=False), pd.read_csv("02-23-2018.csv",low_memory=False)])

In [14]:
web["Label"].value_counts()

Benign              2096222
Brute Force -Web        611
Brute Force -XSS        230
SQL Injection            87
Name: Label, dtype: int64

In [15]:
web = clean_data(web)

Missing Data: 0.349 %
Duplicated Data: 0.43 %


In [16]:
web.head()

,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,Bwd Pkt Len Max,...,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,protocol__0,protocol__6,protocol__17,target
0,20553406,10,7,1063,1297,744,0,106.3,239.357496,976,...,1027304,1027304,1.952608e+07,0.000000e+00,19526080,19526080,0,1,0,0
1,790,2,0,848,0,848,0,424.0,599.626550,0,...,0,0,0.000000e+00,0.000000e+00,0,0,0,1,0,0
2,99745913,5,0,2500,0,500,500,500.0,0.000000,0,...,4000203,4000203,3.191524e+07,3.792787e+07,75584115,7200679,0,0,1,0
3,99745913,5,0,2500,0,500,500,500.0,0.000000,0,...,4000189,4000189,3.191524e+07,3.792788e+07,75584130,7200693,0,0,1,0
4,89481361,6,0,3000,0,500,500,500.0,0.000000,0,...,4000554,4000554,2.137020e+07,1.528109e+07,41990741,7200848,0,0,1,0


In [17]:
web["target"].value_counts()

0    2079841
3        928
Name: target, dtype: int64

### Infiltration Attacks

In [18]:
infl =  pd.concat([pd.read_csv("02-28-2018.csv",low_memory=False), pd.read_csv("03-01-2018.csv",low_memory=False)])

In [19]:
infl = clean_data(infl)

Missing Data: 0.0 %
Duplicated Data: 8.24 %


In [20]:
infl.head()

,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,Bwd Pkt Len Max,...,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,protocol__0,protocol__17,protocol__6,target
0,94658,6,7,708,3718,387,0,118,159.2846508613,1460,...,0,0,0,0,0,0,0,0,1,0
1,206,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,165505,3,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,102429,6,7,708,3718,387,0,118,159.2846508613,1460,...,0,0,0,0,0,0,0,0,1,0
4,167,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [21]:
infl["target"].value_counts()

0    716935
4    135270
Name: target, dtype: int64

### Botnet

In [22]:
botnet = pd.read_csv("03-02-2018.csv",low_memory=False)

In [23]:
botnet["Label"].value_counts()

Benign    762384
Bot       286191
Name: Label, dtype: int64

In [24]:
botnet = clean_data(botnet)

Missing Data: 0.244 %
Duplicated Data: 0.64 %


In [25]:
botnet.head()

,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,Bwd Pkt Len Max,...,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,protocol__0,protocol__6,protocol__17,target
0,141385,9,7,553,3773.0,202,0,61.444444,87.534438,1460,...,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0
1,281,2,1,38,0.0,38,0,19.000000,26.870058,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0
2,279824,11,15,1086,10527.0,385,0,98.727273,129.392497,1460,...,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0
3,132,2,0,0,0.0,0,0,0.000000,0.000000,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0
4,274016,9,13,1285,6141.0,517,0,142.777778,183.887722,1460,...,0.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0


In [26]:
botnet["target"].value_counts()

0    757011
5    282310
Name: target, dtype: int64

### DDoS

In [27]:
# Reading in web attacks but dropping the unnecessary columns Flow ID, Src IP, Src Port and Dst IP
ddos = pd.concat([pd.read_csv("02-20-2018.csv"), pd.read_csv("02-21-2018.csv")])
ddos = ddos.drop(["Flow ID","Src IP","Src Port","Dst IP"], axis=1)

In [28]:
ddos["Label"].value_counts()

Benign                    7733390
DDOS attack-HOIC           686012
DDoS attacks-LOIC-HTTP     576191
DDOS attack-LOIC-UDP         1730
Name: Label, dtype: int64

In [29]:
ddos = clean_data(ddos)

Missing Data: 0.409 %
Duplicated Data: 0.69 %


In [30]:
ddos["target"].value_counts()

0    7652511
6    1246366
Name: target, dtype: int64

Bro you just need to fix the fact that they both have DOS in their names and find away around that shit but otherwise goodstuff.

## Data Set with undersampling

For the undersampling dataset we will balance classes by selecting records of each class equal to the number of classes in the class which appears the least (Web attack class with 928 instances). We will also choose a number of benign classes from each day equal to the proportion of total benign classes they represented in the original dataset.

In [31]:
num_ben_in_one_class / total_number_of_benign_classes_in_original_data_set

NameError: name 'num_ben_in_one_class' is not defined

In [ ]:
bruteforce_sample = bruteforce.loc[bruteforce["target"]==1].sample(n=928).copy()
dos_sample = dos.loc[dos["target"]==2].sample(n=928).copy()
web_sample = web.loc[web["target"]==3].sample(n=928).copy()
infl_sample = infl.loc[infl["target"]==4].sample(n=928).copy()
botnet_sample = botnet.loc[botnet["target"]==5].sample(n=928).copy()
ddos_sample = ddos.loc[ddos["target"]==6].sample(n=928).copy()

In [ ]:
bruteforce_benign = bruteforce.loc[bruteforce["target"]==0].copy()
dos_benign = dos.loc[dos["target"]==0].copy()
web_benign = web.loc[web["target"]==0].copy()
infl_benign = infl.loc[infl["target"]==0].copy()
botnet_benign = botnet.loc[botnet["target"]==0].copy()
ddos_benign = ddos.loc[ddos["target"]==0].copy()

In [ ]:
total_benign = len(bruteforce_benign) + len(dos_benign) + len(web_benign) + len(infl_benign) + len(botnet_benign) + len(ddos_benign)

In [ ]:
bruteforce_ratio = len(bruteforce_benign)/total_benign
dos_ratio = len(dos_benign)/total_benign
web_ratio = len(web_benign)/total_benign
infl_ratio = len(infl_benign)/total_benign
botnet_ratio = len(botnet_benign)/total_benign
ddos_ratio = len(ddos_benign)/total_benign

In [ ]:
bruteforce_benign = bruteforce_benign.sample(n = int(bruteforce_ratio*928))
dos_benign = dos_benign.sample(n = int(dos_ratio*928))
web_benign = web_benign.sample(n = int(web_ratio*928))
infl_benign = infl_benign.sample(n = int(infl_ratio*928))
botnet_benign = botnet_benign.sample(n = int(botnet_ratio*928))
ddos_benign = ddos_benign.sample(n = int(ddos_ratio*928))

To check if the combination has resulted in the correct number of benign class instances

In [ ]:
len(bruteforce_benign) + len(dos_benign) + len(web_benign) + len(infl_benign) + len(botnet_benign) + len(ddos_benign) 

In [ ]:
benign_sample = pd.concat([bruteforce_benign, dos_benign, web_benign, infl_benign, botnet_benign, ddos_benign],axis=0)

In [ ]:
undersampled_dataset = pd.concat([benign_sample,bruteforce_sample, dos_sample, web_sample, infl_sample, botnet_sample, ddos_sample],axis=0).sample(frac = 1)

In [ ]:
sns.countplot(x="target", data = undersampled_dataset)
plt.title("Balanced Classes for CIC-CSE-2018IDS Dataset")
plt.show()

In [ ]:
undersampled_dataset.to_csv("undersampled_cic.csv", index=False)

## Full Data Set